## Load Essential Libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import nltk
import re
from datetime import datetime
from scipy.stats import trim_mean
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

## Load Data

In [ ]:
file_path = 'data/Animation_Movies.csv'
df = pd.read_csv(file_path)
df.head()

# Part 1: Data Recognition

## structure of data

In [ ]:
df.info()
df.describe()

### Calculating Statistical Metrics for Each Numeric Column

In [ ]:
def calculate_statistics(data_frame, Numeric_Features):
    stats__summary = []

    for feature in Numeric_Features:
        dtype = data_frame[feature].dtype
        min_value = data_frame[feature].min()
        max_value = data_frame[feature].max()
        mean_value = data_frame[feature].mean()
        median_value = data_frame[feature].median()
        mode_value = data_frame[feature].mode()[0]  # First mode in case there are multiple modes
        value_range = f"{min_value} - {max_value}"

        # Add the calculated statistics to the summary DataFrame
        stats__summary.append({
            'Feature': feature,
            'Type': dtype,
            'Min': min_value,
            'Max': max_value,
            'Mean': mean_value,
            'Median': median_value,
            'Mode': mode_value,
            'Range': value_range
        })

    # Convert list to DataFrame
    stats_summary_df = pd.DataFrame(stats__summary)

    return stats_summary_df


# List of numerical features for analysis
numeric_features = ['vote_average', 'vote_count', 'runtime', 'revenue', 'budget', 'popularity']
stats_summary = calculate_statistics(df, numeric_features)

# Display the statistical summary table
stats_summary

### Detecting Outliers

In [ ]:
def detect_outliers(data_frame, Numeric_Features):
    outliers_list = []

    for feature in Numeric_Features:
        Q1 = data_frame[feature].quantile(0.25)
        Q3 = data_frame[feature].quantile(0.75)
        IQR = Q3 - Q1
        outliers = data_frame[(data_frame[feature] < (Q1 - 1.5 * IQR)) | (data_frame[feature] > (Q3 + 1.5 * IQR))][
            feature].values

        # Calculate the percentage of outliers
        total_data_points = len(data_frame[feature])
        outlier_count = len(outliers)
        outlier_percentage = (outlier_count / total_data_points) * 100

        # Append a dictionary to the list
        outliers_list.append({
            'Feature': feature,
            'Outliers': outliers,
            'Outlier_Percentage': outlier_percentage
        })

    # Create the summary DataFrame once at the end
    outliers__summary = pd.DataFrame(outliers_list)

    return outliers__summary


# Use the modified function
outliers_summary = detect_outliers(df, numeric_features)

# Display the outliers summary table
outliers_summary

### Creating a Data Recognition Table

In [ ]:
def summarize_data(data_frame, Numeric_Features):
    stats__summary = calculate_statistics(data_frame, Numeric_Features)
    outliers__summary = detect_outliers(data_frame, Numeric_Features)

    # Merge statistics and outliers data
    final__summary = stats__summary.merge(outliers__summary, on='Feature', how='left')
    return final__summary


# Generate the final summary
final_summary = summarize_data(df, numeric_features)

# Display the final summary
final_summary

### Creating a Boxplot for Numerical Data

In [ ]:
def plot_boxplots(data_frame, columns, exclude_zero_columns=None, after_preprocessing=False, figsize=(20, 12),
                  linewidth=1.5):
    if exclude_zero_columns is None:
        exclude_zero_columns = []

    # Determine the number of rows and columns for subplots
    num_plots = len(columns)
    rows = (num_plots + 2) // 3  # Round up to the nearest integer
    cols = 3  # Number of columns

    # Set up the subplots
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    axes = axes.flatten()  # Flatten to easily iterate over

    # Create box-plots for each column
    for i, column in enumerate(columns):
        # Determine the scale of the data for scientific notation
        max_abs_value = data_frame[column].abs().max()
        if max_abs_value != 0:
            exponent = int(np.floor(np.log10(max_abs_value)))
            scale_notation = f'1e{exponent}'
        else:
            scale_notation = '1e0'

        # Apply filters if after_preprocessing is True and if the column is 'runtime'
        if after_preprocessing and column == 'runtime':
            # Filter data to only show points outside the range [10, 90] as potential outliers
            filtered_data = data_frame[(data_frame[column] < 10) | (data_frame[column] > 90)][column]
            sns.boxplot(y=filtered_data, ax=axes[i], linewidth=linewidth, color='cyan')
            axes[i].set_title(f'Boxplot of {column} (Filtered)\nScale: {scale_notation}', fontsize=14)
        else:
            if column in exclude_zero_columns:
                # Separate non-zero data and consider zeros as outliers
                non_zero_data = data_frame[data_frame[column] != 0][column]
                zero_outliers = data_frame[data_frame[column] == 0][column]

                # Plot the box plot for non-zero data
                sns.boxplot(y=non_zero_data, ax=axes[i], linewidth=linewidth, color='cyan')

                # Plot zero values as outliers (by scattering them over the plot)
                axes[i].scatter([0] * len(zero_outliers), zero_outliers, color='red', label='Zero Outliers', marker='x',
                                zorder=5)

                # Add a legend for outliers
                axes[i].legend()
            else:
                # Use the original data for this column
                sns.boxplot(data=data_frame, y=column, ax=axes[i], linewidth=linewidth, color='cyan')

            # Add the title with the scale notation
            axes[i].set_title(f'Boxplot of {column}\nScale: {scale_notation}', fontsize=14)

        axes[i].set_ylabel('Values', fontsize=12)
        axes[i].grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.6)

    # Hide unused subplots if any
    for j in range(num_plots, len(axes)):
        fig.delaxes(axes[j])

    # Adjust the layout for better spacing
    plt.tight_layout()
    plt.show()

In [ ]:
plot_boxplots(df, numeric_features)

# Part2 : Data Quality Assessment

In [ ]:
def data_quality_assessment(df, excluded_columns):
    # Filter columns to be analyzed
    columns_to_analyze = [col for col in df.columns if col not in excluded_columns]

    # Consistency Check
    def check_consistency(data_frame, column):
        if column == 'release_date':
            invalid_dates = data_frame[pd.to_datetime(data_frame[column], errors='coerce').isna()]
            return len(invalid_dates)
        elif column == 'original_language':
            invalid_languages = data_frame[~data_frame[column].str.match(r'^[a-z]{2}$', na=False)]
            return len(invalid_languages)
        elif column == 'status':
            valid_statuses = ['Released', 'In Production', 'Canceled', 'Planned', 'Post Production', 'Rumored']
            invalid_status = data_frame[~data_frame[column].isin(valid_statuses)]
            return len(invalid_status)
        elif column == 'adult':
            valid_adult_values = ['True', 'False']
            invalid_adult = data_frame[~data_frame[column].isin(valid_adult_values)]
            return len(invalid_adult)
        elif column in ['genres', 'production_companies', 'production_countries', 'spoken_languages']:
            invalid_entries = data_frame[column][
                data_frame[column].apply(lambda x: isinstance(x, str) and ',' not in x)]
            return len(invalid_entries)
        else:
            return 'N/A'

    # Currentness Check
    def check_currentness(data_frame, column):
        current_year = datetime.now().year
        if column == 'release_date':
            data_frame['Year'] = pd.to_datetime(data_frame[column], errors='coerce').dt.year
            recent_data = data_frame[data_frame['Year'] >= current_year - 10]
            return len(recent_data)
        elif column == 'status':
            mismatched_status = data_frame[
                ((data_frame['status'] == 'Released') & (
                            pd.to_datetime(data_frame['release_date'], errors='coerce') > datetime.now())) |
                ((data_frame['status'] == 'In Production') & (
                            pd.to_datetime(data_frame['release_date'], errors='coerce') <= datetime.now()))
                ]
            return len(mismatched_status)
        elif column == 'popularity':
            recent_popularity = data_frame[
                (pd.to_datetime(data_frame['release_date'], errors='coerce').dt.year >= current_year - 5) & (
                            data_frame[column] > 0)]
            return len(recent_popularity)
        else:
            return 'N/A'

    # Validity Check
    def check_validity(data_frame, column):
        if column == 'id':
            unique_ids = data_frame['id'].nunique()
            total_records = len(data_frame)
            return total_records - unique_ids
        elif column == 'vote_average':
            invalid_values = data_frame[~data_frame[column].between(0, 10)]
            return len(invalid_values)
        elif column == 'vote_count':
            invalid_values = data_frame[~data_frame[column].apply(lambda x: isinstance(x, int) and x >= 0)]
            return len(invalid_values)
        elif column in ['revenue', 'budget']:
            invalid_values = data_frame[data_frame[column] < 0]
            unreasonable_values = data_frame[data_frame[column] > 1e9]
            return len(invalid_values) + len(unreasonable_values)
        elif column == 'runtime':
            invalid_values = data_frame[(data_frame[column] <= 0) | (data_frame[column] > 300)]
            return len(invalid_values)
        elif column == 'imdb_id':
            invalid_imdb_ids = data_frame[~data_frame[column].str.match(r'^tt\d{7,8}$', na=False)]
            return len(invalid_imdb_ids)
        elif column in ['genres', 'production_companies', 'production_countries', 'spoken_languages']:
            invalid_entries = data_frame[column][
                data_frame[column].apply(lambda x: isinstance(x, str) and ',' not in x)]
            return len(invalid_entries)
        elif column in ['overview', 'tagline']:
            invalid_texts = data_frame[column][
                data_frame[column].str.contains(r'[^\w\s,.!?\'\"-]', regex=True, na=False)]
            return len(invalid_texts)
        else:
            return 'N/A'

    # Completeness Check
    def check_completeness(data_frame, column):
        essential_columns = [
            'id', 'title', 'release_date', 'status', 'vote_count', 'vote_average',
            'popularity', 'budget', 'revenue', 'imdb_id', 'overview', 'genres',
            'production_companies', 'production_countries'
        ]
        if column in essential_columns:
            null_count = data_frame[column].isnull().sum()
            return null_count
        else:
            return 0  # For non-essential columns, assume completeness is not an issue

    # Accuracy Check (now simplified to check data availability)
    def check_accuracy(data_frame, column):
        non_null_count = data_frame[column].notnull().sum()
        accuracy = (non_null_count / len(data_frame)) * 100  # Percentage of non-null values
        return f"{accuracy:.2f}%"

    # Generate the table
    data_quality_table = []

    for column in columns_to_analyze:
        record_count = len(df)
        null_count = check_completeness(df, column)
        consistency_issues = check_consistency(df, column)
        currentness_count = check_currentness(df, column)
        validity_issues = check_validity(df, column)
        accuracy = check_accuracy(df, column)

        data_quality_table.append({
            'Feature Name': column,
            'Number of Records': record_count,
            'Number of Nulls': null_count,
            'Accuracy (%)': accuracy,
            'Completeness': null_count == 0,
            'Validity Issues': validity_issues,
            'Currentness Count': currentness_count,
            'Consistency Issues': consistency_issues
        })

    # Create DataFrame for better visualization
    quality_df = pd.DataFrame(data_quality_table)
    return quality_df


In [ ]:
# Call the function and pass the DataFrame and excluded columns
excluded_columns = ['backdrop_path', 'homepage', 'imdb_id', 'poster_path', 'tagline']
quality_df = data_quality_assessment(df, excluded_columns)

# Display the result
quality_df

# Save the table as a CSV for reporting purposes (optional)
# quality_df.to_csv('/mnt/data/data_quality_report.csv', index=False)

# Part 3: Data Preparation

In [ ]:
df.isnull().sum()

### Drop unused columns from the DataFrame.

In [ ]:
unused_columns = ['backdrop_path', 'homepage', 'imdb_id', 'poster_path', 'tagline']
df.drop(columns=unused_columns, axis=1, inplace=True)

###  Handling Missing, Inconsistent, and Erroneous Data

* #### Replace Blank values

In [ ]:
# Replace Blank values with the string 'null'
df[['title', 'original_title', 'overview']] = df[['title', 'original_title', 'overview']].fillna('null')

* #### release_date 

In [ ]:
def convert_release_date_to_datetime(data_frame):
    """Convert 'release_date' column to datetime format."""
    data_frame['release_date'] = pd.to_datetime(data_frame['release_date'])
    return data_frame


def get_current_date():
    """Get the current date as a timestamp."""
    return pd.Timestamp.now()


def mark_future_releases(data_frame, current_date):
    """Update records with 'release_date' after the current date."""
    future_df = data_frame[data_frame['release_date'] > current_date]

    # Update the status, vote_average, and vote_count for future records
    data_frame.loc[future_df.index, 'vote_count'] = 0
    data_frame.loc[future_df.index, 'vote_average'] = 0
    data_frame.loc[future_df.index, 'status'] = 'In Production'
    return data_frame


def remove_null_release_dates(data_frame):
    """Remove rows where 'release_date' is null."""
    data_frame.dropna(subset=['release_date'], inplace=True)
    return data_frame


def process_dataframe(data_frame):
    """Main function to execute all data processing steps."""
    data_frame = convert_release_date_to_datetime(data_frame)
    current_date = get_current_date()
    data_frame = mark_future_releases(data_frame, current_date)
    data_frame = remove_null_release_dates(data_frame)
    return data_frame

##### Usage

In [ ]:
df = process_dataframe(df)

* #### popularity

In [ ]:
# Replace values greater than 100 with 100
df['popularity'] = df['popularity'].clip(upper=100)

* #### Prediction Model for Filling NaN in Language Field

In [ ]:
def prepare_data(data_frame):
    """Filter rows where 'spoken_languages' and 'production_countries' are not null."""
    return data_frame[data_frame['spoken_languages'].notna() & data_frame['production_countries'].notna()]


def encode_features_and_target(data):
    """Encode 'production_countries' as features and 'spoken_languages' as target."""
    country_encoder = LabelEncoder()
    language_encoder = LabelEncoder()

    features_encoded = country_encoder.fit_transform(data['production_countries']).reshape(-1, 1)
    target_encoded = language_encoder.fit_transform(data['spoken_languages'])

    return features_encoded, target_encoded, country_encoder, language_encoder


def train_model(features_train, target_train):
    """Train a Decision Tree Classifier and return the model."""
    decision_tree_model = DecisionTreeClassifier()
    decision_tree_model.fit(features_train, target_train)
    return decision_tree_model


def predict_missing_languages(data_frame, Model, country_encoder, language_encoder):
    """Predict and fill missing 'spoken_languages' based on 'production_countries'."""
    rows_to_predict = data_frame[data_frame['spoken_languages'].isna() & data_frame['production_countries'].notna()]

    features_to_predict = []
    for country in rows_to_predict['production_countries']:
        try:
            features_to_predict.append(country_encoder.transform([country])[0])
        except ValueError:
            features_to_predict.append(-1)  # Fallback value for unseen labels

    features_to_predict = np.array(features_to_predict).reshape(-1, 1)
    predicted_encoded = Model.predict(features_to_predict)
    predicted_languages = language_encoder.inverse_transform(predicted_encoded)

    data_frame.loc[data_frame['spoken_languages'].isna() & data_frame[
        'production_countries'].notna(), 'spoken_languages'] = predicted_languages
    return data_frame


def fill_null_columns(data_frame):
    """Fill 'production_companies', 'production_countries', 'spoken_languages' columns with 'null' value."""
    data_frame[['production_companies', 'production_countries', 'spoken_languages']] = data_frame[
        ['production_companies', 'production_countries', 'spoken_languages']].fillna('null')
    return data_frame


def run_full_pipeline(data_frame):
    """Run the entire pipeline to prepare data, train the model, predict, and update the DataFrame."""
    # Step 1: Prepare data
    filtered_data = prepare_data(data_frame)

    # Step 2: Encode features and target
    features_encoded, target_encoded, country_encoder, language_encoder = encode_features_and_target(filtered_data)

    # Step 3: Split the data and train the model
    features_train, features_test, target_train, target_test = train_test_split(features_encoded, target_encoded,
                                                                                test_size=0.1, random_state=42)
    classifier = train_model(features_train, target_train)

    # Step 4: Evaluate model accuracy
    predicted_test = classifier.predict(features_test)
    print(f'Model Accuracy: {accuracy_score(target_test, predicted_test) * 100:.3f}%')

    # Step 5: Predict missing values in the original DataFrame
    data_frame = predict_missing_languages(data_frame, classifier, country_encoder, language_encoder)

    # Step 6: Fill null values in specified columns
    data_frame = fill_null_columns(data_frame)

    return data_frame

##### Usage

In [ ]:
df = run_full_pipeline(df)

### Outlier Detection and Removal

In [ ]:
def replace_outliers(data_frame, Numeric_Features, Exclude_Zeros_Columns):
    for feature in Numeric_Features:
        # Ignore zeros if the column is in Exclude_Zeros_Columns
        if feature in Exclude_Zeros_Columns:
            feature_data = data_frame[feature][data_frame[feature] != 0]
        else:
            feature_data = data_frame[feature]

        # Calculate IQR for outlier detection
        Q1 = feature_data.quantile(0.25)
        Q3 = feature_data.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Apply additional filtering for 'runtime' to ignore values between 10 and 90
        if feature == 'runtime':
            lower_bound = min(lower_bound, 10)
            upper_bound = max(upper_bound, 90)

        # Calculate the mean excluding zeros if necessary
        if feature in Exclude_Zeros_Columns:
            mean_value = feature_data.mean()
        else:
            mean_value = data_frame[feature].mean()

        # Replace outliers with the calculated mean
        data_frame[feature] = data_frame[feature].where(
            (data_frame[feature] >= lower_bound) & (data_frame[feature] <= upper_bound),
            mean_value
        )

    return data_frame

##### Usage

In [ ]:
columns_to_process = ['vote_average', 'vote_count', 'runtime', 'revenue', 'budget', 'popularity']
exclude_zeros_columns = ['vote_count', 'revenue', 'budget', 'popularity']
for column in columns_to_process:
    df = replace_outliers(df, columns_to_process, exclude_zeros_columns)

### Text Processing
* #### Stemming, Lemmatization, Stopwords Removal

In [ ]:
# Initialize stopwords, lemmatizer, and stemmer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()


# Function to preprocess text and add a new column
def preprocess_text_column(data_frame, column_name):
    # Create a copy of the DataFrame to avoid modifying the original
    df_copy = data_frame.copy()

    # Define the preprocessing function for each text entry
    def preprocess_text(text):
        # Check if the text is a string; if not, return an empty string
        if not isinstance(text, str):
            return ""

        # Remove special characters and digits
        text = re.sub(r'[^a-zA-Z\s]', '', text)

        # Tokenize the text
        tokens = word_tokenize(text.lower())

        # Remove stopwords and apply lemmatization and stemming
        processed_words = []
        for word in tokens:
            if word not in stop_words:
                lemmatized_word = lemmatizer.lemmatize(word)
                stemmed_word = stemmer.stem(lemmatized_word)
                processed_words.append(stemmed_word)

        # Join processed words back into a single string
        return ' '.join(processed_words)

    # Process the column and add the results as a new column
    processed_column_name = f"processed_{column_name}"
    df_copy[processed_column_name] = df_copy[column_name].apply(preprocess_text)

    return df_copy

##### Usage

In [ ]:
df = preprocess_text_column(df, 'overview')
df[['overview', 'processed_overview']]

### Encoding Categorical Data

In [ ]:
def convert_text_to_numeric(data_frame):
    # 1. Convert 'original_language' to One-Hot Encoding
    data_frame = pd.get_dummies(data_frame, columns=['original_language'], prefix='lang')
    
    # 2. Convert 'genres' to Multi-Label Binary Encoding
    # Split genres by comma and apply One-Hot Encoding
    genres_split = data_frame['genres'].str.get_dummies(sep=',')
    data_frame = pd.concat([data_frame, genres_split], axis=1)
    
    # 3. Convert 'production_companies' to Multi-Label Binary Encoding
    # Split production companies by comma and apply One-Hot Encoding
    companies_split = data_frame['production_companies'].str.get_dummies(sep=',')
    data_frame = pd.concat([data_frame, companies_split], axis=1)
    
    # 4. Convert 'production_countries' to Multi-Label Binary Encoding
    # Split production countries by comma and apply One-Hot Encoding
    countries_split = data_frame['production_countries'].str.get_dummies(sep=',')
    data_frame = pd.concat([data_frame, countries_split], axis=1)
    
    # 5. Convert 'spoken_languages' to Multi-Label Binary Encoding
    # Split spoken languages by comma and apply One-Hot Encoding
    languages_split = data_frame['spoken_languages'].str.get_dummies(sep=',')
    data_frame = pd.concat([data_frame, languages_split], axis=1)
    
    # 6. Convert 'status' to Label Encoding
    # Map each status to an integer
    status_mapping = {'Released': 0, 'In Production': 1, 'Planned': 2, 'Post Production': 3, 'Rumored': 4, 'Canceled': 5}
    data_frame['status'] = data_frame['status'].map(status_mapping)
    
    # 7. Convert 'adult' to binary (0 and 1)
    data_frame['adult'] = data_frame['adult'].map({'False': 0, 'True': 1})
    
    return data_frame

# Example usage
# Assuming 'df' is your DataFrame with the necessary columns
df = convert_text_to_numeric(df)

# Display the first few rows to verify results
df.head(20)

### Converting Numerical Data to Categorical (Discretization)

In [ ]:
def convert_to_categorical(data_frame):
    # 1. Convert 'budget' to categorical
    data_frame['budget_category'] = pd.cut(data_frame['budget'],
                                           bins=[0, 5e7, 1e8, float('inf')],
                                           labels=['Low', 'Medium', 'High'])

    # 2. Convert 'revenue' to categorical
    data_frame['revenue_category'] = pd.cut(data_frame['revenue'],
                                            bins=[0, 5e7, 1.5e8, 5e8, float('inf')],
                                            labels=['Flop', 'Average', 'Hit', 'Blockbuster'])

    # 3. Convert 'popularity' to categorical
    data_frame['popularity_category'] = pd.cut(data_frame['popularity'],
                                               bins=[-float('inf'), 20, 40, 60, float('inf')],
                                               labels=['Low', 'Moderate', 'High', 'Very High'])

    # 4. Convert 'runtime' to categorical
    data_frame['runtime_category'] = pd.cut(data_frame['runtime'],
                                            bins=[0, 40, 90, 120, float('inf')],
                                            labels=['Short', 'Standard', 'Long', 'Very Long'])

    # 5. Convert 'vote_average' to categorical
    data_frame['vote_average_category'] = pd.cut(data_frame['vote_average'],
                                                 bins=[0, 4, 6, 7.5, 10],
                                                 labels=['Poor', 'Average', 'Good', 'Excellent'])

    # 6. Convert 'release_date' to categorical by decade
    # Extract the year from 'release_date' first
    data_frame['year'] = pd.to_datetime(data_frame['release_date'], errors='coerce').dt.year
    # Define categories based on decades
    data_frame['release_decade'] = pd.cut(data_frame['year'],
                                          bins=[1900, 2000, 2010, 2020, float('inf')],
                                          labels=['Before 2000', '2000s', '2010s', '2020s'])

    return data_frame

##### Usage

In [ ]:
df = convert_to_categorical(df)

# Display the first few rows to verify results
df[['budget_category', 'revenue_category', 'popularity_category', 'runtime_category',
    'vote_average_category', 'release_decade']].head()

### Normalization of numerical data

In [ ]:
def normalize_features(df):
    # Initialize scalers
    min_max_scaler = MinMaxScaler()
    standard_scaler = StandardScaler()

    # 1. Normalize 'popularity' with Min-Max scaling (0 to 1)
    df['popularity_normalized'] = min_max_scaler.fit_transform(df[['popularity']])

    # 2. Normalize 'runtime' with Standard scaling (Z-score)
    df['runtime_normalized'] = standard_scaler.fit_transform(df[['runtime']])

    # 3. Normalize 'vote_average' with Standard scaling (Z-score)
    df['vote_average_normalized'] = standard_scaler.fit_transform(df[['vote_average']])

    # 4. Log transformation and Min-Max scaling for 'budget' to reduce skewness
    df['budget_log'] = np.log1p(df['budget'])  # log(1 + x) to handle zero values
    df['budget_normalized'] = min_max_scaler.fit_transform(df[['budget_log']])

    # 5. Log transformation and Min-Max scaling for 'revenue' to reduce skewness
    df['revenue_log'] = np.log1p(df['revenue'])  # log(1 + x) to handle zero values
    df['revenue_normalized'] = min_max_scaler.fit_transform(df[['revenue_log']])

    # 6. Normalize 'vote_count' with Min-Max scaling (0 to 1)
    df['vote_count_normalized'] = min_max_scaler.fit_transform(df[['vote_count']])

    # 7. Min-Max scaling for 'Year' for clustering based on release date
    df['Year_normalized'] = min_max_scaler.fit_transform(df[['Year']])

    # Drop intermediate log-transformed columns to keep the dataset clean
    df.drop(columns=['budget_log', 'revenue_log'], inplace=True)

    return df


df = normalize_features(df)

### Calculate Net Gain 

In [ ]:
df['net_gain'] = df['revenue'] - df['budget']

In [ ]:
df.isnull().sum()

# Part 1 Data Recognition
### Creating a Data Recognition Table

In [ ]:
final_summary = summarize_data(df, columns_to_process)

final_summary

### Creating a box plot for numerical data

In [ ]:
plot_boxplots(df, columns_to_process, exclude_zeros_columns, after_preprocessing=True)

## Part1.3 (association inspection)

###  Budget and Revenue Scatter plot and association inspection

In [ ]:
# Assuming your DataFrame 'data_frame' contains 'budget' and 'revenue' columns

# Scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(df['budget'], df['revenue'], color='blue', alpha=0.5)
plt.title('Relationship between Revenue and Budget')
plt.xlabel('Budget')
plt.ylabel('Revenue')
plt.grid(True)

# Fit a linear regression model
X = df['budget'].values.reshape(-1, 1)  # Reshape for sklearn
y = df['revenue'].values

model = LinearRegression()
model.fit(X, y)

# Predict using the model
y_pred = model.predict(X)

# Plot the best-fit line
plt.plot(df['budget'], y_pred, color='red', linewidth=2, label='Best-fit line')

# Show plot with best-fit line
plt.legend()
plt.show()

# Calculate Pearson association_score coefficient
association_score = df['budget'].corr(df['revenue'])
print(f'Correlation between Budget and Revenue: {association_score:.2f}')

# Calculate R² score (accuracy of the line)
r2 = r2_score(y, y_pred)
print(f'R² score (accuracy of the line): {r2:.2f}')


### vote_average and vote_count  histogram inspection

In [ ]:


# Assuming your DataFrame 'data_frame' contains 'vote_average' and 'vote_count' columns

# Create bins for vote_average
bins = 10
df['vote_average_binned'] = pd.cut(df['vote_average'], bins=bins)

# Group by the binned vote_average and sum the vote_count
binned_data = df.groupby('vote_average_binned')['vote_count'].sum().reset_index()

# Plotting the bar chart
plt.figure(figsize=(10, 6))
plt.bar(binned_data['vote_average_binned'].astype(str), binned_data['vote_count'], color='blue', alpha=0.7)
plt.title('Vote Count by Vote Average')
plt.xlabel('Vote Average')
plt.ylabel('Vote Count')
plt.xticks(rotation=45)  # Rotate x labels for better visibility
plt.grid(axis='y', alpha=0.75)
plt.show()


### Release date Analysis

In [ ]:


# Sample DataFrame
# data_frame = pd.read_csv('your_data.csv')  # Load your DataFrame

# Step 1: Convert release_date to datetime format
df['release_date'] = pd.to_datetime(df['release_date'])

# Step 2: Extract the year from the release_date
df['year'] = df['release_date'].dt.year

# Step 3: Filter the DataFrame based on the desired status (replace 'desired_status' with your status value)
desired_status = 'Released'  # Replace with the status you are interested in
filtered_data_frame = df[df['status'] == desired_status]

# Step 4: Group by year and count the records
yearly_counts = filtered_data_frame['year'].value_counts().sort_index()

# Step 5: Plot the results
plt.figure(figsize=(10, 6))
plt.plot(yearly_counts.index, yearly_counts.values, marker='o')
plt.title(f"Number of Records Released Each Year with Status '{desired_status}'")
plt.xlabel("Year")
plt.ylabel("Number of Records")
plt.grid()
plt.xticks(yearly_counts.index)  # Show all years on the x-axis
plt.show()


### Genres inspection

In [ ]:
# Step 1: Split the 'genres' column into separate rows
data_frame_genres = df.assign(genres=df['genres'].str.split(',')).explode('genres')

# Step 2: Remove any extra spaces around genres
data_frame_genres['genres'] = data_frame_genres['genres'].str.strip()

# Step 3: Group by genres and calculate the mean vote_average and total revenue for each genre
genre_stats = data_frame_genres.groupby('genres').agg(
    total_movies=('genres', 'size'),
    average_vote=('vote_average', 'mean'),
    total_revenue=('revenue', 'sum')
).reset_index()

# Step 4: Sort by total_movies or total_revenue to find the most popular genres
genre_stats_sorted = genre_stats.sort_values(by=['total_movies', 'total_revenue'], ascending=False)

# Display the result
print(genre_stats_sorted)


### production_companies inspection

In [ ]:
# Step 1: Split 'production_companies' column into separate rows
data_frame_companies = df.assign(production_companies=df['production_companies'].str.split(',')).explode(
    'production_companies')

# Step 2: Remove any extra spaces around company names
data_frame_companies['production_companies'] = data_frame_companies['production_companies'].str.strip()

# Step 3: Filter movies with high ratings (e.g., vote_average >= 7.0)
high_rated_movies = data_frame_companies[data_frame_companies['vote_average'] >= 7.0]

# Step 4: Group by production companies and calculate metrics
company_stats = high_rated_movies.groupby('production_companies').agg(
    num_high_rated_movies=('production_companies', 'size'),  # Number of high-rated movies
    avg_popularity=('popularity', 'mean'),  # Average popularity of movies
    avg_revenue=('revenue', 'mean'),  # Average revenue of movies
    avg_vote=('vote_average', 'mean')  # Average vote rating of movies
).reset_index()

# Step 5: Sort by number of high-rated movies or revenue (optional)
company_stats_sorted = company_stats.sort_values(by=['num_high_rated_movies', 'avg_revenue'], ascending=False)

# Display the result
pd.DataFrame(company_stats_sorted).head(7)

### adult inspection

In [ ]:
# Step 1: Separate adult and non-adult movies
adult_movies = df[df['adult'] == True]
non_adult_movies = df[df['adult'] == False]

# Step 2: Calculate metrics for adult movies
adult_stats = adult_movies.agg({
    'revenue': 'mean',
    'vote_average': 'mean',
    'popularity': 'mean'
})

# Step 3: Calculate metrics for non-adult movies
non_adult_stats = non_adult_movies.agg({
    'revenue': 'mean',
    'vote_average': 'mean',
    'popularity': 'mean'
})

# Step 4: Combine the results for comparison
comparison_stats = pd.DataFrame({
    'Metric': ['Revenue', 'Vote Average', 'Popularity'],
    'Adult': [adult_stats['revenue'], adult_stats['vote_average'], adult_stats['popularity']],
    'Non-Adult': [non_adult_stats['revenue'], non_adult_stats['vote_average'], non_adult_stats['popularity']]
})

# Display the result
pd.DataFrame(comparison_stats)

### budget/revenue inspection

In [ ]:
# Remove rows where the budget is zero or where revenue or budget data is missing (NaN values)
data_frame_clean = df.dropna(subset=['revenue', 'budget'])

# budget can't be 0
data_frame_clean = data_frame_clean[data_frame_clean['budget'] > 0]

# Create a new column for the revenue-to-budget ratio
data_frame_clean['revenue_to_budget'] = data_frame_clean['revenue'] / data_frame_clean['budget']

# Sort the movies based on the revenue-to-budget ratio in descending order
data_frame_sorted = data_frame_clean.sort_values(by='revenue_to_budget', ascending=False)

# Display the top 10 movies in terms of financial success
top_10_successful_movies = data_frame_sorted[['title', 'revenue', 'budget', 'revenue_to_budget']].head(10)

pd.DataFrame(top_10_successful_movies)


### spoken languages analysis

In [ ]:
# Step 1: Clean and split the 'spoken_languages' column
# Remove quotes and split the languages by comma
df['spoken_languages'] = df['spoken_languages'].str.replace("'", "").str.split(', ')

# Step 2: Explode the list to separate rows for each language
data_frame_exploded = df.explode('spoken_languages')

# Step 3: Count the occurrences of each language
language_counts = data_frame_exploded['spoken_languages'].value_counts()

In [ ]:
# Display the results
pd.DataFrame(language_counts).head(10)

### popularity vs vote_average inspection

In [ ]:


# Step 1: Calculate the association_score between 'runtime' and 'budget'
association_score = df['vote_average'].corr(df['popularity'])
print(f'Correlation between runtime and budget: {association_score}')

# Step 2: Create a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(df['vote_average'], df['popularity'], color='blue', alpha=0.7)
plt.title('Scatter Plot of vote_average vs popularity')
plt.xlabel('vote_average ')
plt.ylabel('popularity')
plt.grid(True)
plt.show()


###  vote_count vs vote_average inspection

In [ ]:


# Step 1: Calculate the association_score between 'runtime' and 'budget'
association_score = df['vote_count'].corr(df['vote_average'])
print(f'Correlation between runtime and budget: {association_score}')

# Step 2: Create a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(df['vote_count'], df['vote_average'], color='blue', alpha=0.7)
plt.title('Scatter Plot of vote_count vs vote_average')
plt.xlabel('vote_count ')
plt.ylabel('vote_average')
plt.grid(True)
plt.show()


### runtime and budget association inspection

In [ ]:


# Step 1: Calculate the association_score between 'runtime' and 'budget'
association_score = df['runtime'].corr(df['budget'])
print(f'Correlation between runtime and budget: {association_score}')

# Step 2: Create a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(df['runtime'], df['budget'], color='blue', alpha=0.7)
plt.title('Scatter Plot of Runtime vs Budget')
plt.xlabel('Runtime (minutes)')
plt.ylabel('Budget (currency units)')
plt.grid(True)
plt.show()


## Filter the records that are after 2014  (last 10 years ago)

In [ ]:
# Step 1: Convert 'release_date' to datetime format
df['release_date'] = pd.to_datetime(df['release_date'])

# Step 2: Filter the DataFrame to drop rows with 'release_date' less than 2014-01-01
data_frame_filtered = df[df['release_date'] >= '2014-01-01']

# Display the updated DataFrame
data_frame_filtered.head()

### Check how many  records that have budget=0 or null and revenue != 0 or not null

In [ ]:
# Condition to check budget = 0 or null and revenue != 0 or not null (consistency)
condition = (df['budget'].isnull() | (df['budget'] == 0)) & (df['revenue'].notnull() & (df['revenue'] != 0))

# Count the number of records that satisfy the condition
count = df[condition].shape[0]
print(count)